# Playing with Color
stough 202-

In this activity we'll use what we've learned about color to try and **enhance** or even **compress** them without too much perceptual loss. The other notebooks in this folder should be useful here, particularly [color_intro](./color_intro.ipynb), [color_HSV](./color_HSV.ipynb), and [color_YCbCr](./color_YCbCr.ipynb) 

## Imports
You can borrow the imports from any of the other notebooks in this directory.

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatSlider

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import arr_info
from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       lab_uniform)

## 1. Enhancement: Improving the Color of an Image
Take a look at  `../dip_pics/sf.jpg` and try the `vis_hists` function on it to the distributions of the R,G,B channels. Of course you can try this out with any image you like of course.

In [2]:
I1 = plt.imread('../dip_pics/sf.jpg')
vis_hists(I1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
arr_info(I1)

((947, 1421, 3), dtype('uint8'), 0, 255)

In [4]:
plt.figure()
plt.imshow(I1 >> 2)
bin((255>>4)<<4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

'0b11110000'

In [5]:
len(np.unique(I1.ravel()))

256

In [6]:
J = I1 >> 4
len(np.unique(J.ravel()))

16

In [7]:
plt.imshow((I1>>6)<<6) ## Not correctly normalized

In [8]:
J1 = (I1 >> 5)
J1 = np.uint8(J1*(255/7))
plt.imshow(J1)

In [9]:
vis_hists(J1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

To me it looks like the clouds shrouding the [Golden Gate Bridge](https://en.wikipedia.org/wiki/Golden_Gate_Bridge) are too dark, not blue enough really for my tastes. You can see this in the relative color distributions as well, with the blue component maxing out below the midpoint of the intensity range. 

Try adding some blue to the image to really make those clouds pop better. **Be careful** to use the `arr_info` to understand both the `dtype` and understood range of your data. 

In [10]:
Img = I1.copy()
Img2 = I1.copy()
Img[...,2] = Img[...,2]*1.5
arr_info(Img)
Img2[...,1] = Img2[...,1]*1.3


In [11]:
f, ax = plt.subplots(1,3,figsize=(12,4), sharex=True, sharey=True)
ax[0].imshow(I1)
ax[0].set_title('Raw image')
ax[1].imshow(Img) # black to red colormap
ax[1].set_title('Blue enhanced')
ax[2].imshow(Img2)
ax[2].set_title('Green enhanced')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Green enhanced')

## 2. Ehancement: Improving Contrast through Saturation of Color
Take a look at `../dip_pics/world_overexposed.jpg`. Here we have a very overexposed image. (Again, find any overexposed image you like). What this means generally is that we have a lot of what should be color/Hue showing up as mostly white; as in, it's been mixed with too much white. See the [color_HSV](./color_HSV.ipynb) demo titled "Viewing the planes of the HSV space" and see what happens as saturation $S_{HSV}$ approaches zero. 

Try converting the image to HSV and then scaling the saturation component to improve the color purity of all those washed-out pixels. Then convert back to RGB and see if it's improved. **Remember**, from the [skimage documentation](https://scikit-image.org/docs/dev/api/skimage.color.html#skimage.color.rgb2hsv), that the hsv channels are all in $[0,1]$, so when you do the scaling you might need to `np.clip`. 

In [12]:
Irgb = plt.imread('../dip_pics/world_overexposed.jpg')
vis_hists(Irgb)
Ihsv = color.rgb2hsv(Irgb)
vis_hists(Ihsv)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
arr_info(Irgb)

((960, 1280, 3), dtype('uint8'), 31, 255)

In [14]:
In = Irgb - Irgb.min()
In = In/In.max()
In = (255*In).astype(np.uint8)

arr_info(In)

((960, 1280, 3), dtype('uint8'), 0, 255)

In [15]:
f, ax = plt.subplots(1,2,figsize=(12,3),sharex=True,sharey=True)
ax[0].imshow(Irgb)
ax[1].imshow(In)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
HSV = color.rgb2hsv(Irgb)
H, S, V = np.split(HSV, indices_or_sections=3, axis=-1)

S2 = S.copy() / S.max()
S2 = (S2 + .1).clip(0,1)
arr_info(S2)

nS = S
nS[nS > 0.2] = nS[nS > 0.2] + 1.0

HSV = np.dstack((H,nS,V))
I_schanged = color.hsv2rgb(HSV)

f, ax = plt.subplots(1,2,figsize=(12,3),sharex=True,sharey=True)
ax[0].imshow(Irgb)
ax[1].imshow(I_schanged)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


## 3. Compression: Reducing the Bit-Depth of an Image to Save Space
In a prior activity we reduced the bit-depth of the RGB color channels of an image and saw how that *quantization* affected the image quality, creating sharp edges of color change in what would be smoothly graded areas. So one of the limits of potentially compressing images relates to the minimum bit-depth that can maintain perceptual fidelity.

But in this module's materials, we've seen that some color spaces (HSV, YCbCr, and L*a*b*) include some dimension that captures the brightness/luminance/intensity representation of an image separately from the "color" aspect. [YCbCr](./color_YCbCr.ipynb) in particular is used a lot in compression for precisely this reason.

In the below cells, try considering an image like `../dip_pics/bellagio.jpg` or `../dip_pics/mountainSpring.jpg` for example. 

- See what happens with you reduce the bit-depth to 4-4-4 in RGB. 
- Then try reducing the bit-depth to 6-3-3 but **in the YCbCr** space. Here we're saving the same number of bits per pixel, but differentially quantizing (and therefore compressing) the **Luminance** and color components.

It may be useful here to define a function that quantizes a 2D `ndarray` instead of a whole image at once. Luckily, it's likely the same quantization function you've already written may suffice. I include a useable version here just to help.  

**Be careful**: [`skimage.color.ycbcr2rgb`](https://scikit-image.org/docs/dev/api/skimage.color.html#skimage.color.ycbcr2rgb) expects 'float64' data type. 

In [34]:
I3 = plt.imread('../dip_pics/bellagio.jpg')
vis_hists(I3)
arr_info(I3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

((687, 1024, 3), dtype('uint8'), 0, 255)

In [35]:
def quantize_bitdepth(C, bitdepth=8):
    """
    quantize_bitdepth(C, bitdepth=8): return a copy of the array C where the elements are
    limited to 2**bitdepth unique values. Tries to handle a float image just in case.
    """
    assert bitdepth <= 8, f'quantize_bitdepth error: expects bitdepth <= 8, got {bitdepth}'
    
    shft_amt = 8 - bitdepth
    
    J = np.uint8(C)
    if C.dtype == 'float':
        J = np.uint8(256 * (J/J.max()))
        
    return np.uint8((J >> shft_amt)*(255/(2**bitdepth-1)))

In [36]:
def quantize_bitdepth_float(C, bitdepth=8):
    """
    quantize_bitdepth(C, bitdepth=8): return a copy of the array C where the elements are
    limited to 2**bitdepth unique values. Tries to handle a float image just in case.
    """
    assert bitdepth <= 8, f'quantize_bitdepth error: expects bitdepth <= 8, got {bitdepth}'
    
    shft_amt = 8 - bitdepth
    
    return (C >> shft_amt)*(255/(2**bitdepth-1))

In [37]:
Img = I3.copy()
Img[...,0] = quantize_bitdepth(Img[...,0],4)
Img[...,2] = quantize_bitdepth(Img[...,2],4)
Img[...,1] = quantize_bitdepth(Img[...,1],4)
arr_info(Img[...,2])
vis_hists(Img)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
f, ax = plt.subplots(1,2,figsize=(12,8),sharex=True,sharey=True)
ax[0].imshow(I3)
ax[0].set_title('Original')
ax[1].imshow(Img)
ax[1].set_title('4-4-4 RGB')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '4-4-4 RGB')

In [29]:
Iybr = color.rgb2ycbcr(I3)

Y, Cb, Cr = np.split(Iybr,indices_or_sections=3, axis=-1)
Y2 = Y.copy()
Cb2 = Cb.copy()
Cr2 = Cr.copy()

In [30]:
arr_info(Iybr)

((687, 1024, 3), dtype('float64'), 16.0, 229.07583529411764)

In [31]:
Jybr = Iybr.copy()
Jybr[...,0] = quantize_bitdepth(Jybr[...,0],6)
Jybr[...,1] = quantize_bitdepth(Jybr[...,1],3)
Jybr[...,2] = quantize_bitdepth(Jybr[...,2],3)

In [32]:
arr_info(Jybr[...,2])

((687, 1024), dtype('float64'), 0.0, 255.0)

In [33]:
plt.figure()
plt.imshow(color.ycbcr2rgb(Jybr.astype('float64')))

vis_hists(color.ycbcr2rgb(Jybr.astype('float64')))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
